In [1]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
import pandas as pd

gs = GridSearchCV(SVR(),
                  param_grid=[{'kernel':['rbf'], 'gamma':[0.1, 0.01]},
                              {'kernel':['poly'], 'degree':[2, 3]}])

X, y = make_classification(random_state=0)
pd.DataFrame(gs.fit(X, y).search_results_)

,estimator_score_test_mean,estimator_score_test_rank,estimator_score_test_split_0,estimator_score_test_split_1,estimator_score_test_split_2,estimator_score_test_std,param_degree,param_gamma,param_kernel
0,0.270179,3,7.731211,9.098761,10.187935,1.005097,NaN,0.1,rbf
1,0.473821,1,17.931207,14.106300,15.344608,1.593522,NaN,0.01,rbf
2,-0.379306,4,-21.095093,-8.870404,-7.965071,5.987581,2,NaN,poly
3,0.302800,2,9.500712,7.369996,13.409303,2.500895,3,NaN,poly


In [2]:
gs.grid_scores_

/home/rvraghav93/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:729: DeprecationWarning: The grid_scores_ attribute is deprecated in favor of the more elaborate search_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.27018, std: 1.00510, params: {'kernel': 'rbf', 'gamma': 0.1},
 mean: 0.47382, std: 1.59352, params: {'kernel': 'rbf', 'gamma': 0.01},
 mean: -0.37931, std: 5.98758, params: {'kernel': 'poly', 'degree': 2},
 mean: 0.30280, std: 2.50089, params: {'kernel': 'poly', 'degree': 3}]

In [3]:
from sklearn.grid_search import GridSearchCV as GridSearchCVo
gso = GridSearchCVo(SVR(), param_grid=[{'kernel':['rbf'], 'gamma':[0.1, 0.01]},
                                       {'kernel':['poly'], 'degree':[2, 3]}])
gso.fit(X, y).grid_scores_

/home/rvraghav93/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/cross_validation.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/rvraghav93/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


[mean: 0.27018, std: 0.03340, params: {'kernel': 'rbf', 'gamma': 0.1},
 mean: 0.47382, std: 0.04121, params: {'kernel': 'rbf', 'gamma': 0.01},
 mean: -0.37931, std: 0.17260, params: {'kernel': 'poly', 'degree': 2},
 mean: 0.30280, std: 0.07655, params: {'kernel': 'poly', 'degree': 3}]

In [4]:
# Neither of the following two estimators inherit from BaseEstimator,
# to test hyperparameter search on user-defined classifiers.
class MockClassifier(object):
    """Dummy classifier to test the parameter search algorithms"""
    def __init__(self, foo_param=0):
        self.foo_param = foo_param

    def fit(self, X, Y):
        return self

    def predict(self, T):
        return T.shape[0]

    predict_proba = predict
    decision_function = predict
    transform = predict

    def score(self, X=None, Y=None):
        if self.foo_param > 1:
            score = 1.
        else:
            score = 0.
        return score

    def get_params(self, deep=False):
        return {'foo_param': self.foo_param}

    def set_params(self, **params):
        self.foo_param = params['foo_param']
        return self

# Test that the best estimator contains the right value for foo_param
clf = MockClassifier()
grid_search = GridSearchCV(clf, {'foo_param': [1, 2, 3]})
# make sure it selects the smallest parameter in case of ties
grid_search.fit(X, y)
grid_search.best_estimator_.foo_param == 2

True

In [5]:
pd.DataFrame(grid_search.search_results_)

,estimator_score_test_mean,estimator_score_test_rank,estimator_score_test_split_0,estimator_score_test_split_1,estimator_score_test_split_2,estimator_score_test_std,param_foo_param
0,0.0,3,0.0,0.0,0.0,0.000000,1
1,1.0,1,34.0,33.0,33.0,0.471405,2
2,1.0,2,34.0,33.0,33.0,0.471405,3


In [6]:
import numpy as np
np.argsort(grid_search.search_results_["estimator_score_test_mean"], kind='heapsort')[::-1]

array([1, 2, 0])

In [7]:
grid_search.best_estimator_.foo_param

2

In [8]:
from sklearn.datasets import make_blobs
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score

# test that correct scores are used
clf = LinearSVC(random_state=0)
X, y = make_blobs(random_state=0, centers=2)
Cs = [.1, 1, 10]
for score in ['f1', 'roc_auc']:
    grid_search = GridSearchCV(clf, {'C': Cs}, scoring=score)
    grid_search.fit(X, y)
    cv = StratifiedKFold(n_folds=3)
    for C, scores in zip(Cs, grid_search.grid_scores_):
        clf.set_params(C=C)
        scores = scores[2]  # get the separate runs from grid scores
        i = 0
        for train, test in cv.split(X, y):
            clf.fit(X[train], y[train])
            if score == "f1":
                correct_score = f1_score(y[test], clf.predict(X[test]))
            elif score == "roc_auc":
                dec = clf.decision_function(X[test])
                correct_score = roc_auc_score(y[test], dec)
            
            print correct_score, scores[i]
            assert correct_score == scores[i]
            i += 1

/home/rvraghav93/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:729: DeprecationWarning: The grid_scores_ attribute is deprecated in favor of the more elaborate search_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


0.969696969697 32.9696969697


AssertionError: 

In [11]:
pd.DataFrame(grid_search.search_results_)

,f1_test_mean,f1_test_rank,f1_test_split_0,f1_test_split_1,f1_test_split_2,f1_test_std,param_C
0,0.911374,3,32.969697,27.200000,30.967742,2.391957,0.1
1,0.923296,2,33.028571,28.333333,30.967742,1.921585,1
2,0.924340,1,33.028571,29.405405,30.000000,1.586510,10


In [21]:
def foobar(a, b=5, c=4):
    print a, b, c
    return 5
    
def boobar():
    return 5

In [13]:
from functools import partial

In [24]:
foobar_with_a_set = partial(foobar, a=5)

In [25]:
for fn in [foobar_with_a_set, boobar]:
    assert 5 == fn()

5 5 4


In [19]:
foobar(6, c=3)

6 5 3
